# Mean-Variance Portfolio Optimization

## Problem Statement
Construct an optimal portfolio that balances expected return against risk (variance), subject to realistic constraints.

## Markowitz Framework

**Objective**: Maximize Sharpe ratio
$$\max_w \frac{w^T \mu - r_f}{\sqrt{w^T \Sigma w}}$$

**Constraints**:
- Portfolio weights sum to 1: $\sum_i w_i = 1$
- Long-only: $w_i \geq 0$
- Position limits: $w_i \leq w_{\max}$
- Sector constraints

Alternative formulations:
1. **Minimum variance**: $\min_w w^T \Sigma w$
2. **Risk parity**: $w_i (\Sigma w)_i = \text{const}$
3. **Maximum Sharpe**: Optimize risk-adjusted returns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

np.random.seed(42)

## 1. Generate Synthetic Market Data

In [ ]:
# Portfolio parameters
n_assets = 20
asset_names = [f"Asset_{i+1}" for i in range(n_assets)]

# Assign sectors
sectors = ['Tech', 'Finance', 'Energy', 'Healthcare', 'Consumer']
asset_sectors = np.random.choice(sectors, n_assets)

# Generate expected returns (annual %)
mu = np.random.uniform(0.05, 0.20, n_assets)  # 5% to 20% expected returns

# Generate correlation matrix
# Higher correlation within sectors
def generate_covariance_matrix(n_assets, sectors):
    # Base correlation
    base_corr = 0.3
    sector_corr = 0.6
    
    # Correlation matrix
    corr = np.eye(n_assets) * (1 - base_corr) + base_corr
    
    # Increase correlation within sectors
    for i in range(n_assets):
        for j in range(i+1, n_assets):
            if sectors[i] == sectors[j]:
                corr[i, j] = corr[j, i] = sector_corr
    
    # Generate volatilities
    vols = np.random.uniform(0.15, 0.40, n_assets)  # 15% to 40% annual vol
    
    # Convert to covariance matrix
    D = np.diag(vols)
    cov = D @ corr @ D
    
    return cov, vols

Sigma, vols = generate_covariance_matrix(n_assets, asset_sectors)

# Risk-free rate
rf = 0.03  # 3% risk-free rate

# Create DataFrame
asset_data = pd.DataFrame({
    'Asset': asset_names,
    'Sector': asset_sectors,
    'Expected Return': mu * 100,
    'Volatility': vols * 100
})

print("Asset Universe:")
print(asset_data.to_string(index=False))
print(f"\nRisk-free rate: {rf*100:.1f}%")

## 2. Efficient Frontier

In [ ]:
def portfolio_stats(weights, mu, Sigma, rf):
    """
    Compute portfolio return, volatility, and Sharpe ratio.
    """
    portfolio_return = np.dot(weights, mu)
    portfolio_vol = np.sqrt(np.dot(weights, np.dot(Sigma, weights)))
    sharpe = (portfolio_return - rf) / portfolio_vol
    return portfolio_return, portfolio_vol, sharpe

def minimize_volatility(target_return, mu, Sigma):
    """
    Find minimum variance portfolio for target return.
    """
    n = len(mu)
    
    # Objective: minimize variance
    def objective(w):
        return np.dot(w, np.dot(Sigma, w))
    
    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
        {'type': 'eq', 'fun': lambda w: np.dot(w, mu) - target_return}  # target return
    ]
    
    # Bounds: long-only
    bounds = tuple((0, 1) for _ in range(n))
    
    # Initial guess: equal weights
    w0 = np.ones(n) / n
    
    result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result.x if result.success else None

# Compute efficient frontier
min_return = np.min(mu)
max_return = np.max(mu)
target_returns = np.linspace(min_return, max_return * 0.95, 50)

frontier_vols = []
frontier_returns = []

for target in target_returns:
    weights = minimize_volatility(target, mu, Sigma)
    if weights is not None:
        ret, vol, _ = portfolio_stats(weights, mu, Sigma, rf)
        frontier_returns.append(ret)
        frontier_vols.append(vol)

frontier_returns = np.array(frontier_returns)
frontier_vols = np.array(frontier_vols)

## 3. Optimal Portfolios

In [ ]:
def max_sharpe_portfolio(mu, Sigma, rf):
    """
    Find portfolio with maximum Sharpe ratio.
    """
    n = len(mu)
    
    # Objective: maximize Sharpe (minimize negative Sharpe)
    def neg_sharpe(w):
        ret, vol, sharpe = portfolio_stats(w, mu, Sigma, rf)
        return -sharpe
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = tuple((0, 1) for _ in range(n))
    w0 = np.ones(n) / n
    
    result = minimize(neg_sharpe, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def min_variance_portfolio(mu, Sigma):
    """
    Find minimum variance portfolio (global minimum on frontier).
    """
    n = len(mu)
    
    def objective(w):
        return np.dot(w, np.dot(Sigma, w))
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = tuple((0, 1) for _ in range(n))
    w0 = np.ones(n) / n
    
    result = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def constrained_portfolio(mu, Sigma, rf, max_weight=0.10, sector_limits=None):
    """
    Maximum Sharpe with position and sector constraints.
    """
    n = len(mu)
    
    def neg_sharpe(w):
        ret, vol, sharpe = portfolio_stats(w, mu, Sigma, rf)
        return -sharpe
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Add sector constraints if provided
    if sector_limits is not None:
        for sector, limit in sector_limits.items():
            sector_mask = (asset_sectors == sector).astype(float)
            constraints.append({
                'type': 'ineq',
                'fun': lambda w, mask=sector_mask, lim=limit: lim - np.dot(w, mask)
            })
    
    bounds = tuple((0, max_weight) for _ in range(n))
    w0 = np.ones(n) / n
    
    result = minimize(neg_sharpe, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Compute optimal portfolios
w_max_sharpe = max_sharpe_portfolio(mu, Sigma, rf)
w_min_var = min_variance_portfolio(mu, Sigma)
w_constrained = constrained_portfolio(mu, Sigma, rf, max_weight=0.10, 
                                      sector_limits={'Tech': 0.30, 'Finance': 0.25})

# Equal weight benchmark
w_equal = np.ones(n_assets) / n_assets

portfolios = {
    'Max Sharpe': w_max_sharpe,
    'Min Variance': w_min_var,
    'Constrained': w_constrained,
    'Equal Weight': w_equal
}

# Display statistics
print("\nPortfolio Comparison:")
print(f"\n{'Portfolio':<20} {'Return %':<12} {'Vol %':<12} {'Sharpe':<12}")
print("-" * 60)

for name, weights in portfolios.items():
    ret, vol, sharpe = portfolio_stats(weights, mu, Sigma, rf)
    print(f"{name:<20} {ret*100:<12.2f} {vol*100:<12.2f} {sharpe:<12.3f}")

## 4. Visualizations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Top-left: Efficient frontier
axes[0, 0].plot(frontier_vols * 100, frontier_returns * 100, 'b-', linewidth=2, label='Efficient Frontier')

# Plot individual assets
axes[0, 0].scatter(vols * 100, mu * 100, s=80, alpha=0.6, c='gray', edgecolors='black', label='Assets')

# Plot optimal portfolios
for name, weights in portfolios.items():
    ret, vol, _ = portfolio_stats(weights, mu, Sigma, rf)
    marker = 'o' if name != 'Equal Weight' else 'x'
    axes[0, 0].scatter(vol * 100, ret * 100, s=150, marker=marker, label=name, edgecolors='black', linewidths=2)

# Capital market line (for max Sharpe)
ret_ms, vol_ms, sharpe_ms = portfolio_stats(w_max_sharpe, mu, Sigma, rf)
x_cml = np.array([0, vol_ms * 1.5])
y_cml = rf + sharpe_ms * x_cml
axes[0, 0].plot(x_cml * 100, y_cml * 100, 'r--', linewidth=2, label='Capital Market Line', alpha=0.7)

axes[0, 0].set_xlabel('Volatility (%)', fontsize=12)
axes[0, 0].set_ylabel('Expected Return (%)', fontsize=12)
axes[0, 0].set_title('Mean-Variance Efficient Frontier', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=9, loc='upper left')
axes[0, 0].grid(True, alpha=0.3)

# Top-right: Portfolio weights comparison
portfolio_names = ['Max Sharpe', 'Min Variance', 'Constrained']
x = np.arange(n_assets)
width = 0.25

for i, name in enumerate(portfolio_names):
    weights = portfolios[name]
    axes[0, 1].bar(x + i * width, weights * 100, width, label=name, alpha=0.8)

axes[0, 1].set_xlabel('Assets', fontsize=12)
axes[0, 1].set_ylabel('Weight (%)', fontsize=12)
axes[0, 1].set_title('Portfolio Weights Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x + width)
axes[0, 1].set_xticklabels([f"A{i+1}" for i in range(n_assets)], rotation=45, ha='right')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Bottom-left: Sector allocation (Max Sharpe portfolio)
sector_weights = {}
for sector in sectors:
    mask = asset_sectors == sector
    sector_weights[sector] = np.sum(w_max_sharpe[mask])

colors = sns.color_palette('husl', len(sectors))
axes[1, 0].pie(sector_weights.values(), labels=sector_weights.keys(), autopct='%1.1f%%',
               colors=colors, startangle=90, textprops={'fontsize': 11})
axes[1, 0].set_title('Max Sharpe: Sector Allocation', fontsize=14, fontweight='bold')

# Bottom-right: Correlation heatmap
corr_matrix = Sigma / np.outer(vols, vols)
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, vmin=-1, vmax=1, 
            square=True, ax=axes[1, 1], cbar_kws={'label': 'Correlation'})
axes[1, 1].set_title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks([])
axes[1, 1].set_yticks([])

plt.tight_layout()
plt.savefig('portfolio_optimization.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Risk Decomposition

In [ ]:
def risk_contribution(weights, Sigma):
    """
    Compute marginal and total risk contribution of each asset.
    """
    portfolio_var = np.dot(weights, np.dot(Sigma, weights))
    portfolio_vol = np.sqrt(portfolio_var)
    
    # Marginal contribution to risk (MCR)
    mcr = np.dot(Sigma, weights) / portfolio_vol
    
    # Total contribution to risk
    tcr = weights * mcr
    
    # Percentage contribution
    pct_contribution = tcr / portfolio_vol * 100
    
    return mcr, tcr, pct_contribution

# Analyze Max Sharpe portfolio
mcr, tcr, pct_contrib = risk_contribution(w_max_sharpe, Sigma)

risk_df = pd.DataFrame({
    'Asset': asset_names,
    'Weight %': w_max_sharpe * 100,
    'Risk Contrib %': pct_contrib
}).sort_values('Risk Contrib %', ascending=False)

print("\nRisk Contribution Analysis (Max Sharpe Portfolio):")
print(risk_df.head(10).to_string(index=False))

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
top_10 = risk_df.head(10)
x_pos = np.arange(len(top_10))

ax.bar(x_pos, top_10['Weight %'], alpha=0.7, label='Weight %', color='steelblue')
ax.bar(x_pos, top_10['Risk Contrib %'], alpha=0.7, label='Risk Contrib %', color='coral')

ax.set_xlabel('Assets', fontsize=12)
ax.set_ylabel('Percentage', fontsize=12)
ax.set_title('Weight vs Risk Contribution (Top 10 Assets)', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(top_10['Asset'], rotation=45, ha='right')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('risk_contribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Key Insights

**Markowitz Optimization Challenges**:
1. **Estimation error**: Covariance matrix estimates are noisy → unstable weights
2. **Concentration**: Optimal portfolios often highly concentrated (few assets)
3. **Corner solutions**: Weights at bounds (0 or max) common
4. **Turnover**: Small changes in inputs → large weight changes

**Solutions**:
- **Regularization**: Add penalty term $\lambda ||w||_2^2$ to objective
- **Shrinkage estimators**: Ledoit-Wolf covariance estimation
- **Robust optimization**: Min-max formulations
- **Bayesian methods**: Black-Litterman model
- **Resampling**: Bootstrap efficient frontier

**Advanced Techniques**:
1. **Risk parity**: Equalize risk contribution across assets
2. **Hierarchical clustering**: Group correlated assets first
3. **Factor models**: Optimize on factor exposures
4. **Transaction costs**: Include $c |w_t - w_{t-1}|$ in objective

**Production Considerations**:
- Daily rebalancing with turnover constraints
- Bid-ask spreads and market impact
- Tax implications (capital gains)
- Regulatory constraints (position limits)